# Week 2: Iterators, Generators, Decorators, Caching and Memoization

## Iterators

### What is iteration?

Iteration is the process of traversing a series of objects, handling each one in turn until there are no more objects to deal with.

For example, a for loop represents a type of iteration.

In python, we often iterate through a list:

```
numbers = [1,3,5,7,9]
for number in numbers:
    print(number)
```

### Iterable vs. iterator

* in python, an iterable object is one that implements an `__iter__` method
* in python, an iterator is an object that implements a `__next__` method

* iter() returns an iterator for an object
* next() returns the next element from an iterator (by invoking the underlying `__next__` method)
* usually the case that a class has both an ```__iter__``` and a ```__next__``` method

In [1]:
years = [1967, 1974, 1955, 2029]
years_iter = iter(years)
print(next(years_iter))
print(next(years_iter))
print(next(years_iter))
print(next(years_iter))
next(years_iter)

1967
1974
1955
2029


StopIteration: 

In [2]:
for year in years:
    print(year)

1967
1974
1955
2029


### Python's `for` loop is really a `while` loop with an iterator!
```
for x in some_iterable:
    do_something_with(x)
```

is really

```
iterator = iter(some_iterable)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        break
    yield item
```

### Let's define a class that's iterable and that returns an iterator
(that is, one that returns an interator when ```__iter__``` is called)

In [3]:
class Decades:
    def __init__(self, years):
        self.years = years
        self.index = 0
    def __iter__(self):
        return self # note that self has a __next__ method, so it's an iterator
    def __next__(self):
        if self.index >= len(self.years):
            raise StopIteration
        retval = self.years[self.index] // 10 * 10
        self.index += 1
        return retval


In [4]:
decades = Decades([1971,1982,2019])

In [5]:
decades_iterator = iter(decades)

In [6]:
next(decades_iterator)

1970

## Generators

### Generators are similar to functions, so let's start there:

Let's look at a simple function that returns a list of n elements, each of which is a string containing "Hello"

In [7]:
def Hello(n = 0):
    ret = []
    for i in range(n):
        ret.append("Hello")
    return ret

In [8]:
hellos = Hello(5)

In [9]:
for hello in hellos:
    print(hello)

Hello
Hello
Hello
Hello
Hello


## Our first generator


In [10]:
def HelloGenerator(max = 0):
    x = 0
    while True:
        if x < max:
            yield "Hello"
            x = x+1
        else:
            break


In [11]:
hellos = HelloGenerator(5)

In [12]:
for hello in hellos:
    print(hello)

Hello
Hello
Hello
Hello
Hello


### Generators vs. Functions
* most important difference is that generators use the ```yield``` statement, whereas functions use ```return```
* ```yield``` returns a value, stops executing the code at that point and maintains state until it's called again
* when invoked, returns an object but doesn't start executing code
* implements ```__iter__``` and ```__next__``` automatically (hey, that's useful!)

### List comprehensions vs. generator expressions

In [13]:
# Initialize the list, in this case with a list of years
year_list = [2018, 1776, 2020, 1977, 1980, 2009, 2019]

# Find the decade corresponding to each of the years
decade_list = [x//10*10 for x in year_list]

In [14]:
# same thing with a generator
decade_generator = (x//10*10 for x in year_list)

In [15]:
max(decade_list)

2020

In [16]:
min(decade_generator)

1770

## Filtering

In the following code:

In [17]:
decade_generator_filtered = (x//10*10 for x in year_list if x > 1900)

the parentheses ('()') create a generator expression

In [18]:
max(decade_generator_filtered)

2020

## Memory size issues

In [19]:
import sys

In [20]:
sys.getsizeof(decade_list)

120

In [21]:
sys.getsizeof(decade_generator)

112

In [22]:
big_year_list = [x for x in range(1770,2020)]
big_decade_list = [x//10*10 for x in big_year_list]

In [23]:
sys.getsizeof(big_decade_list)

2200

In [24]:
big_decade_generator = (x//10*10 for x in big_year_list)

In [25]:
sys.getsizeof(big_decade_generator)

112

## A more complex example

In [26]:
def lines_words_chars(text):
    yield ('lines',len(text.splitlines()))
    yield ('words',len(text.split()))
    yield ('characters',len(text))

In [27]:
a = lines_words_chars("This is a text")

In [28]:
next(a)

('lines', 1)

In [29]:
next(a)

('words', 4)

In [30]:
next(a)

('characters', 14)

In [31]:
next(a)

StopIteration: 

In [32]:
# skip all non-lowercased letters (including punctuation)
# append 1 if lowercase letter is "o"
# append 0 if lowercase letter is not "o"
out = []
for i in "Hello. How Are You?":
    if i.islower():
        out.append(1 if i is "o" else 0)

In [33]:
out

[0, 0, 0, 1, 1, 0, 0, 0, 1, 0]

In [34]:
# NOTE: this is not efficient because statistics.mean() will create a list from a generator
#       before proceeding with the calculation




from statistics import mean
out2 = mean(1 if char is 'o' else 0 for char in "Hello. How Are You?" if char.islower())
out2

0.3

## Let's take a look at some python code:

From https://github.com/python/cpython/blob/master/Lib/statistics.py


```
# === Measures of central tendency (averages) ===
def mean(data):
    """Return the sample arithmetic mean of data.
    >>> mean([1, 2, 3, 4, 4])
    2.8
    >>> from fractions import Fraction as F
    >>> mean([F(3, 7), F(1, 21), F(5, 3), F(1, 3)])
    Fraction(13, 21)
    >>> from decimal import Decimal as D
    >>> mean([D("0.5"), D("0.75"), D("0.625"), D("0.375")])
    Decimal('0.5625')
    If ``data`` is empty, StatisticsError will be raised.
    """
    if iter(data) is data:
        data = list(data)
    n = len(data)
    if n < 1:
        raise StatisticsError('mean requires at least one data point')
    T, total, count = _sum(data)
    assert count == n
    return _convert(total/n, T)
```

In [35]:
data = [1,2,3]
idata = iter(data)

In [36]:
data

[1, 2, 3]

In [37]:
idata

In [38]:
def dgen():
    yield 1
    yield 2
    yield 3

In [39]:
dg = dgen()

In [40]:
dgi = iter(dg)

In [41]:
dg

<generator object dgen at 0x72f4fc272270>

In [42]:
dgi

<generator object dgen at 0x72f4fc272270>

In [43]:
if data is not idata:
    print(1)

1


In [44]:
if iter(data) is data:
    print("yes")

In [45]:
data

[1, 2, 3]

In [46]:
iter(data)

## Memoization

A common way to teach memoization is to use Fibonacci numbers, defined as 

$ F_{0}=0,\quad F_{1}=1,$

and 

$ F_{n}=F_{n-1}+F_{n-2},$

for $n > 1$

Thus, the first few Fibonacci numbers are:
0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55...

Here's an implementation of the code to calculate Fibonacci numbers:

### LEARNING CHECK POSSIBILITY: GET THEM TO WRITE A FIBONACCI FUNCTION

In [47]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [48]:
fibonacci(35)

9227465

### Important digression: Jupyter magic commands
* sometimes you'll see a line in a Jupyter notebook that starts with a '%'
* these are "magic" commands
* we'll deal with these in more detail in a later lecture, but for now we're going to introduce %timeit
* %timeit will tell you how much time a line takes to run
* %%timeit will tell you how much time a cell takes to run

In [49]:
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

In [50]:
%timeit fibonacci(32)

760 ms ± 3.83 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Caching
* typically done with web browsers and web pages
* let's take a look at a simple caching example

## Memoization: a special form of caching
* caching is a more general approach: e.g. web pages
* memozation is caching of the output of a function given a specific set of parameters

## Memoization example: 

In [51]:
def memoize(func):
    cache = dict()

    def memoized_func(*args):
        if args in cache:
            return cache[args]
        result = func(*args)
        cache[args] = result
        return result

    return memoized_func

In [52]:
memoized_fibonacci = memoize(fibonacci)

In [53]:
%timeit memoized_fibonacci(32)

The slowest run took 12.83 times longer than the fastest. This could mean that an intermediate result is being cached.
716 ns ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
%timeit memoized_fibonacci(30)

The slowest run took 11.46 times longer than the fastest. This could mean that an intermediate result is being cached.
673 ns ± 957 ns per loop (mean ± std. dev. of 7 runs, 1 loop each)


Note that the memoized version doesn't call the memoized verision when it recurses.

## Important Digression: Decorators
* recall example from above where one function (memoized_fibonacci) returned another function (fibonacci)
* this is a specific form of a more general approach called decorators
* let's take a look at the simplest form of a decorator, the null decorator, which does nothing

In [55]:
def null_decorator(func):
    return func

In [56]:
def hello():
    return "Hello"

In [57]:
hello()

'Hello'

In [58]:
decorated_hello = null_decorator(hello)

In [59]:
decorated_hello()

'Hello'

### Now let's look at a slightly more complicated example that takes some function (assuming it returns a string) and wraps the output in ```<em>...</em>``` tags

In [60]:
def emphasize(func):
    def wrapper():
        original_ret = func()
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper


In [61]:
emphasized_hello = emphasize(hello)

In [62]:
emphasized_hello()

'<em>Hello</em>'

### Using the @: wrapping functions simplified
* commonly referred to as "syntactic sugar", the @ command allows you to wrap a function with one line


In [63]:
def emphasize(func):
    def wrapper():
        original_ret = func()
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper


In [64]:
@emphasize
def hello():
    return "Hello"

In [65]:
hello()

'<em>Hello</em>'

## A slightly more complicated example: decorating functions that take parameters
* let's say we have a function that returns "Hello" in some specified language:

In [66]:
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [67]:
multilingual_hello('en')

'Hello'

In [68]:
multilingual_hello('fr')

'Bonjour'

## And now let's say we want to decorate that function with our emphasize wrapper:

In [69]:
@emphasize
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [70]:
multilingual_hello()

'<em>Hello</em>'

In [71]:
multilingual_hello('fr')

TypeError: wrapper() takes 0 positional arguments but 1 was given

### Uh oh... what just happened?
* our wrapper isn't set up to take any paramters, but our underlying function expects one (optional) one
* we can change our decorator to accommodate the optional paramter by using ```*args``` and ```**kwargs```:

In [72]:
def emphasize_args(func):
    def wrapper(*args,**kwargs):
        original_ret = func(*args,**kwargs)
        modified_ret = "<em>" + original_ret + "</em>"
        return modified_ret
    return wrapper

In [73]:
@emphasize_args
def multilingual_hello(lang = 'en'):
    lookup = {'en':'Hello','fr':'Bonjour'}
    return lookup[lang]

In [74]:
multilingual_hello('fr')

'<em>Bonjour</em>'

## Ok, back to memoization
* but first, another digression: functools
* functools: https://docs.python.org/3/library/functools.html
* "Higher-order functions and operations on callable objects"
* of note, ``` @functools.lru_cache ```


In [75]:
import functools

@functools.lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

Or equivalently:

In [76]:
from functools import lru_cache

@lru_cache(maxsize=128)
def fibonacci(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    return fibonacci(n - 1) + fibonacci(n - 2)

See also https://en.wikipedia.org/wiki/Cache_replacement_policies#Least_recently_used_(LRU) for LRU

In [77]:
%timeit fibonacci(10)

71.8 ns ± 1.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [78]:
%timeit fibonacci(20)

71.9 ns ± 0.226 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [79]:
%timeit fibonacci(30)

72.3 ns ± 1.43 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [80]:
%timeit fibonacci(40)

71.4 ns ± 0.205 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)
